# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /Users/Joerg 1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Joerg 1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('messages_merged', 'sqlite:///pipeline_project.db') 
X = df.message.values
Y = df.iloc[:,4:]
categories = Y.columns
Y=Y.values

#https://knowledge.udacity.com/questions/56317
#several 2 values in first column of df > needs to be cleaned
for i, col in enumerate(Y):
    for j, row in enumerate(col):
        if col[j] not in [0, 1]:
            Y[i][j] = 1

In [3]:
df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products  ...  \
0        0      0            0             0                 0  ...   
1        0      0            1             0                 0  ...   
2        0      0            0             0                 0  ...   
3        1      0            1             0                 1  ...   
4        0      0            0             0                 0  ...   

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0                0       0      0     0   
1            0                     0                1       0      1     0   
2            0                     0                0       0      0     0   
3            0                     0                0       0      0     0   
4            0                     0                0       0      0     0   

   earthquake  cold  other_weather  direct_report  
0           0     0              0              0  
1           0     0              0              0  
2           0     0              0              0  
3           0     0              0              0  
4           0     0              0              0  

[5 rows x 40 columns]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = re.sub(r"[^A-Za-z0-9\-]", " ", text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=1), n_jobs=1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred=pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(categories)))

                        precision    recall  f1-score   support

               related       0.81      0.85      0.83      5048
               request       0.46      0.45      0.45      1103
                 offer       0.00      0.00      0.00        29
           aid_related       0.58      0.54      0.56      2755
          medical_help       0.25      0.22      0.23       522
      medical_products       0.19      0.18      0.19       315
     search_and_rescue       0.18      0.17      0.18       186
              security       0.04      0.04      0.04       122
              military       0.30      0.23      0.26       217
           child_alone       0.00      0.00      0.00         0
                 water       0.52      0.43      0.47       453
                  food       0.57      0.50      0.53       744
               shelter       0.46      0.40      0.43       598
              clothing       0.15      0.13      0.14       101
                 money       0.21      

/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_f

In [11]:
y_pred=pipeline.predict(X_test)
#print(classification_report(y_test, y_pred, target_names=list(categories)))
labels=np.array(list(categories))
target_names=list(categories)
                
for i in range(len(categories)):
    print(classification_report(y_test[:, i], y_pred[:, i]))


              precision    recall  f1-score   support

           0       0.41      0.35      0.38      1506
           1       0.81      0.85      0.83      5048

   micro avg       0.74      0.74      0.74      6554
   macro avg       0.61      0.60      0.61      6554
weighted avg       0.72      0.74      0.73      6554

              precision    recall  f1-score   support

           0       0.89      0.89      0.89      5451
           1       0.46      0.45      0.45      1103

   micro avg       0.82      0.82      0.82      6554
   macro avg       0.67      0.67      0.67      6554
weighted avg       0.82      0.82      0.82      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

              preci

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      5295
           1       0.44      0.40      0.42      1259

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.65      0.64      0.64      6554
weighted avg       0.78      0.79      0.78      6554



### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
#https://medium.com/all-things-ai/in-depth-parameter-tuning-for-random-forest-d67bb7e920d
parameters = {
    'clf__estimator__n_estimators' : [8, 16, 32]

}

cv = GridSearchCV(pipeline, cv=5, param_grid=parameters)

cv.fit(X_train, y_train)

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [56]:
y_pred=cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(categories)))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      5014
               request       0.90      0.40      0.56      1129
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.60      0.67      2691
          medical_help       0.57      0.05      0.09       508
      medical_products       0.79      0.03      0.06       340
     search_and_rescue       0.88      0.08      0.15       180
              security       0.00      0.00      0.00       113
              military       0.67      0.06      0.11       205
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.15      0.26       414
                  food       0.85      0.35      0.50       759
               shelter       0.86      0.22      0.35       555
              clothing       0.50      0.03      0.05       109
                 money       1.00      

/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_f

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.